# MovieLens: Spark-based Big Data Recommendation Analysis

## Task4- CONTRASTING EXPERIMENT TWO PREDICTION METHOD COMPARISON

## Model2-Part1-Feature Engineering
---

In [1]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# 1. Load Movies Data

In [3]:
movies = pd.read_csv("./ml-25m/movies.csv")
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
movies['genres'] = movies['genres'].apply(lambda x: x.split("|"))
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II (1995),[Comedy]


In [17]:
from collections import Counter

genres_counts = Counter(g for genres in movies['genres'] for g in genres)
print(f"There are {len(genres_counts)} genre labels.")
genres_counts

There are 20 genre labels.


Counter({'Adventure': 4136,
         'Animation': 2927,
         'Children': 2930,
         'Comedy': 16844,
         'Fantasy': 2724,
         'Romance': 7708,
         'Drama': 25559,
         'Action': 7335,
         'Crime': 5314,
         'Thriller': 8631,
         'Horror': 5970,
         'Mystery': 2921,
         'Sci-Fi': 3583,
         'IMAX': 195,
         'Documentary': 5561,
         'War': 1874,
         'Musical': 1053,
         'Western': 1399,
         'Film-Noir': 353,
         '(no genres listed)': 4798})

In [18]:
movies = movies[movies['genres']!='(no genres listed)']

del genres_counts['(no genres listed)']

## 1.1. Get movies' year

In [6]:
import re

def extract_year_from_title(title):
    match = re.search(r'\(\d{4}\)', title)
    if match:
        year = match.group(0).strip('()')
        return int(year)
    return None

title = "Toy Skokie\xa0(1981)"
year = extract_year_from_title(title)
print(f"Year of release: {year}")
print(type(year))

Year of release: 1981
<class 'int'>


In [7]:
movies['year'] = movies['title'].apply(extract_year_from_title)
movies.head()

,movieId,title,genres,year
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1995.0
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]",1995.0
2,3,Grumpier Old Men (1995),"[Comedy, Romance]",1995.0
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]",1995.0
4,5,Father of the Bride Part II (1995),[Comedy],1995.0


In [8]:
movies['year'].nunique()

135

In [9]:
print(f"Original number of movies: {movies['movieId'].nunique()}")

Original number of movies: 62423


In [10]:
movies = movies[~movies['year'].isnull()]
print(f"Number of movies after removing null years: {movies['movieId'].nunique()}")

Number of movies after removing null years: 62013


## 1.2. Get movies' decade

In [11]:
x = 1995

def get_decade(year):
    year = str(year)
    decade_prefix = year[0:3] # get first 3 digits of year
    decade = f'{decade_prefix}0' # append 0 at the end
    return int(decade)

get_decade(x)

1990

In [13]:
def round_down(year):
    return year - (year%10)

round_down(x)

1990

In [14]:
movies['decade'] = movies['year'].apply(round_down)

In [19]:
genres = list(genres_counts.keys())

for g in genres:
    movies[g] = movies['genres'].transform(lambda x: int(g in x))

In [20]:
movies[genres].head()

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir
0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Great! Our genres columns are represented as binary feautres. The next step is to wrangle our `decade` column so that each decade has its own column. We can do this using pandas' [get_dummies()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html) function, which works by creating a categorical variable into binary variables.

In [21]:
movie_decades = pd.get_dummies(movies['decade'])
movie_decades.head()

,1870.0,1880.0,1890.0,1900.0,1910.0,1920.0,1930.0,1940.0,1950.0,1960.0,1970.0,1980.0,1990.0,2000.0,2010.0
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [22]:
movie_features = pd.concat([movies['movieId'],movies['title'],movies[genres], movie_decades], axis=1)
movie_features.head()

,movieId,title,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,...,1920.0,1930.0,1940.0,1950.0,1960.0,1970.0,1980.0,1990.0,2000.0,2010.0
0,1,Toy Story (1995),1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,2,Jumanji (1995),1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Grumpier Old Men (1995),0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Waiting to Exhale (1995),0,0,0,1,0,1,1,0,...,0,0,0,0,0,0,0,1,0,0
4,5,Father of the Bride Part II (1995),0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


# 2. Output movie's feature

In [53]:
movie_features.to_csv("./movies_feature.csv")

movie_features

,movieId,title,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,...,1920.0,1930.0,1940.0,1950.0,1960.0,1970.0,1980.0,1990.0,2000.0,2010.0
0,1,Toy Story (1995),1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,2,Jumanji (1995),1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Grumpier Old Men (1995),0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Waiting to Exhale (1995),0,0,0,1,0,1,1,0,...,0,0,0,0,0,0,0,1,0,0
4,5,Father of the Bride Part II (1995),0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62418,209157,We (2018),0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
62419,209159,Window of the Soul (2001),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
62420,209163,Bad Poems (2018),0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
62421,209169,A Girl Thing (2001),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


# 3. Load ratings' data

In [54]:
df_1=pd.read_csv('./ml-25m/ratings.csv')

In [57]:
len(df_1)

25000095

# 4. Merge ratings.csv and movies feature

In [61]:
import pandas as pd

# 使用 merge 方法根据 movieId 合并数据框
merged_df = pd.merge(train, movie_features, on='movieId', how='left')

# merged_df 现在包含 train_df 的数据和 movie_df 中对应 movieId 的数据
print(merged_df.head())

   userId  movieId  rating   timestamp                      title  Adventure  \
0   92471    86833     4.0  1564933259         Bridesmaids (2011)        0.0   
1   77400     7836     3.5  1150312388           Woodstock (1970)        0.0   
2   12996     3597     1.0   974950287             Whipped (2000)        0.0   
3   71689      590     5.0   834606725  Dances with Wolves (1990)        1.0   
4   54052     1359     2.5  1520069745  Jingle All the Way (1996)        0.0   

   Animation  Children  Comedy  Fantasy  ...  1920.0  1930.0  1940.0  1950.0  \
0        0.0       0.0     1.0      0.0  ...     0.0     0.0     0.0     0.0   
1        0.0       0.0     0.0      0.0  ...     0.0     0.0     0.0     0.0   
2        0.0       0.0     1.0      0.0  ...     0.0     0.0     0.0     0.0   
3        0.0       0.0     0.0      0.0  ...     0.0     0.0     0.0     0.0   
4        0.0       1.0     1.0      0.0  ...     0.0     0.0     0.0     0.0   

   1960.0  1970.0  1980.0  1990.0  200

# 5. Generate users' data

In [16]:
def generate_user_features(X_train):
    # Calculate the average rating for each user
    avg_rating = (X_train.groupby('userId').agg({'rating': np.mean})
                  .rename(columns={'rating': 'user_avg_rating'}))

    avg_rating_squared = (avg_rating['user_avg_rating'] * avg_rating['user_avg_rating']).to_frame()
    avg_rating_squared.columns = ['user_avg_rating_squared']
    
    min_rating = (X_train.groupby('userId').agg({'rating': np.min})
                  .rename(columns={'rating': 'user_min_rating'}))
    max_rating = (X_train.groupby('userId').agg({'rating': np.max})
                  .rename(columns={'rating': 'user_max_rating'}))
    median_rating = (X_train.groupby('userId').agg({'rating': np.median})
                     .rename(columns={'rating': 'user_median_rating'}))
    range_rating = (X_train.groupby('userId').agg({'rating': lambda x: x.max() - x.min()})
                    .rename(columns={'rating': 'user_rating_range'}))
    std_rating = (X_train.groupby('userId').agg({'rating': np.std})
                  .rename(columns={'rating': 'user_rating_std'}))
    
    num_movies = (X_train.groupby('userId').agg({'rating': lambda x: x.size})
                  .rename(columns={'rating': 'user_num_movies_rated'}))
    
    num_movies_squared = (num_movies['user_num_movies_rated'] * num_movies['user_num_movies_rated']).to_frame()
    num_movies_squared.columns = ['user_num_movies_rated_squared']
    
    return pd.concat([avg_rating, avg_rating_squared, min_rating, max_rating, median_rating, 
                      range_rating, std_rating, num_movies, num_movies_squared], axis=1)


## Merge data

In [4]:
df=pd.read_csv('./ml-25m/ratings.csv')

In [7]:
print(df.head())

   userId  movieId  rating   timestamp
0       1      296     5.0  1147880044
1       1      306     3.5  1147868817
2       1      307     5.0  1147868828
3       1      665     5.0  1147878820
4       1      899     3.5  1147868510


In [8]:
movies = pd.read_csv("./ml-25m/movies.csv")
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [9]:
{g for s in movies['genres'] for g in s.split('|')}

{'(no genres listed)',
 'Action',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'IMAX',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western'}

In [10]:
movies['genres'] = movies['genres'].apply(lambda x: x.split("|"))
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II (1995),[Comedy]


In [12]:
import pandas as pd

# 使用 merge 方法根据 movieId 合并数据框
df = pd.merge(df, movies, on='movieId', how='left')

# merged_df 现在包含 train_df 的数据和 movie_df 中对应 movieId 的数据
print(df.head())

   userId  movieId  rating   timestamp  \
0       1      296     5.0  1147880044   
1       1      306     3.5  1147868817   
2       1      307     5.0  1147868828   
3       1      665     5.0  1147878820   
4       1      899     3.5  1147868510   

                                              title  \
0                               Pulp Fiction (1994)   
1  Three Colors: Red (Trois couleurs: Rouge) (1994)   
2  Three Colors: Blue (Trois couleurs: Bleu) (1993)   
3                                Underground (1995)   
4                        Singin' in the Rain (1952)   

                             genres  
0  [Comedy, Crime, Drama, Thriller]  
1                           [Drama]  
2                           [Drama]  
3              [Comedy, Drama, War]  
4        [Comedy, Musical, Romance]  


In [17]:
df_1=generate_user_features(df)

In [18]:
print(df_1.head())

        user_avg_rating  user_avg_rating_squared  user_min_rating  \
userId                                                              
1              3.814286                14.548776              0.5   
2              3.630435                13.180057              0.5   
3              3.697409                13.670830              2.0   
4              3.378099                11.411554              0.5   
5              3.752475                14.081070              2.0   

        user_max_rating  user_median_rating  user_rating_range  \
userId                                                           
1                   5.0                 4.0                4.5   
2                   5.0                 4.0                4.5   
3                   5.0                 4.0                3.0   
4                   5.0                 3.5                4.5   
5                   5.0                 4.0                3.0   

        user_rating_std  user_num_movies_rated  user_

In [26]:
df_2=pd.read_csv("./movies_feature.csv")
df_2 = df_2.drop('Unnamed: 0', axis=1)
print(df_2.head())

   movieId                               title  Adventure  Animation  \
0        1                    Toy Story (1995)          1          1   
1        2                      Jumanji (1995)          1          0   
2        3             Grumpier Old Men (1995)          0          0   
3        4            Waiting to Exhale (1995)          0          0   
4        5  Father of the Bride Part II (1995)          0          0   

   Children  Comedy  Fantasy  Romance  Drama  Action  ...  1920.0  1930.0  \
0         1       1        1        0      0       0  ...       0       0   
1         1       0        1        0      0       0  ...       0       0   
2         0       1        0        1      0       0  ...       0       0   
3         0       1        0        1      1       0  ...       0       0   
4         0       1        0        0      0       0  ...       0       0   

   1940.0  1950.0  1960.0  1970.0  1980.0  1990.0  2000.0  2010.0  
0       0       0       0       0   

In [25]:
df=pd.read_csv('./ml-25m/ratings.csv')

In [27]:
print(df.head())

   userId  movieId  rating   timestamp
0       1      296     5.0  1147880044
1       1      306     3.5  1147868817
2       1      307     5.0  1147868828
3       1      665     5.0  1147878820
4       1      899     3.5  1147868510


In [28]:
# 使用 merge 方法根据 movieId 合并数据框
df_3 = pd.merge(df, df_1, on='userId', how='left')

In [29]:
print(df_3.head())

   userId  movieId  rating   timestamp  user_avg_rating  \
0       1      296     5.0  1147880044         3.814286   
1       1      306     3.5  1147868817         3.814286   
2       1      307     5.0  1147868828         3.814286   
3       1      665     5.0  1147878820         3.814286   
4       1      899     3.5  1147868510         3.814286   

   user_avg_rating_squared  user_min_rating  user_max_rating  \
0                14.548776              0.5              5.0   
1                14.548776              0.5              5.0   
2                14.548776              0.5              5.0   
3                14.548776              0.5              5.0   
4                14.548776              0.5              5.0   

   user_median_rating  user_rating_range  user_rating_std  \
0                 4.0                4.5         1.004235   
1                 4.0                4.5         1.004235   
2                 4.0                4.5         1.004235   
3               

In [30]:
# 使用 merge 方法根据 movieId 合并数据框
df_3 = pd.merge(df_3, df_2, on='movieId', how='left')
print(df_3.head())

   userId  movieId  rating   timestamp  user_avg_rating  \
0       1      296     5.0  1147880044         3.814286   
1       1      306     3.5  1147868817         3.814286   
2       1      307     5.0  1147868828         3.814286   
3       1      665     5.0  1147878820         3.814286   
4       1      899     3.5  1147868510         3.814286   

   user_avg_rating_squared  user_min_rating  user_max_rating  \
0                14.548776              0.5              5.0   
1                14.548776              0.5              5.0   
2                14.548776              0.5              5.0   
3                14.548776              0.5              5.0   
4                14.548776              0.5              5.0   

   user_median_rating  user_rating_range  ...  1920.0  1930.0  1940.0 1950.0  \
0                 4.0                4.5  ...     0.0     0.0     0.0    0.0   
1                 4.0                4.5  ...     0.0     0.0     0.0    0.0   
2                 4.

In [31]:
df_3.to_csv('./final.csv')

In [32]:
print(df_3.columns)

Index(['userId', 'movieId', 'rating', 'timestamp', 'user_avg_rating',
       'user_avg_rating_squared', 'user_min_rating', 'user_max_rating',
       'user_median_rating', 'user_rating_range', 'user_rating_std',
       'user_num_movies_rated', 'user_num_movies_rated_squared', 'title',
       'Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy', 'Romance',
       'Drama', 'Action', 'Crime', 'Thriller', 'Horror', 'Mystery', 'Sci-Fi',
       'IMAX', 'Documentary', 'War', 'Musical', 'Western', 'Film-Noir',
       '1870.0', '1880.0', '1890.0', '1900.0', '1910.0', '1920.0', '1930.0',
       '1940.0', '1950.0', '1960.0', '1970.0', '1980.0', '1990.0', '2000.0',
       '2010.0'],
      dtype='object')


In [3]:
print(df_3.head(5))

   Unnamed: 0  userId  movieId  rating   timestamp  user_avg_rating  \
0           0       1      296     5.0  1147880044         3.814286   
1           1       1      306     3.5  1147868817         3.814286   
2           2       1      307     5.0  1147868828         3.814286   
3           3       1      665     5.0  1147878820         3.814286   
4           4       1      899     3.5  1147868510         3.814286   

   user_avg_rating_squared  user_min_rating  user_max_rating  \
0                14.548776              0.5              5.0   
1                14.548776              0.5              5.0   
2                14.548776              0.5              5.0   
3                14.548776              0.5              5.0   
4                14.548776              0.5              5.0   

   user_median_rating  ...  1920.0  1930.0  1940.0  1950.0 1960.0  1970.0  \
0                 4.0  ...     0.0     0.0     0.0     0.0    0.0     0.0   
1                 4.0  ...     0.0